### **Tutorial de Hugging Face**

Hugging Face proporciona acceso a modelos (tanto el código que los implementa como sus pesos preentrenados, incluyendo los últimos LLMs como Llama3, DBRX, etc.), tokenizadores específicos de los modelos, así como pipelines para tareas comunes de NLP, y datasets y métricas en un paquete separado llamado `datasets`. Tiene implementaciones en PyTorch, Tensorflow y Flax.

Vamos a repasar algunos casos de uso:

* Descripción general de Tokenizers y modelos
* Ajuste fino. Usaremos un ejemplo de clasificación de sentimientos.

Se pueden aplicar a otros proyectos interesantes tambien:

1. Aplicar un modelo preentrenado existente a una nueva aplicación o tarea y explorar cómo abordarlo/solucionarlo.
2. Implementar una nueva o compleja arquitectura neural y demostrar su rendimiento en algunos datos.
3. Analizar el comportamiento de un modelo: cómo representa el conocimiento lingüístico o qué tipo de fenómenos puede manejar o errores que comete.

De estos, `transformers` será de mayor ayuda para (1) y para (3). (2) implica una forma muy conveniente diseñar un modelo basado en los existentes proporcionados por Hugging Face. No lo cubriremos aquí y por favor revisa a [este ejemplo](https://huggingface.co/docs/transformers/en/custom_models).

Aquí hay recursos adicionales que introducen la librería que se utilizaron para hacer este cuaderno:

* [Docs de Hugging Face](https://huggingface.co/docs/transformers/index)
  * Documentación clara
  * Tutoriales, recorridos y cuadernos de ejemplo
  * Lista de modelos disponibles
* [Curso de Hugging Face](https://huggingface.co/course/)
* [Ejemplos de Hugging Face](https://github.com/huggingface/transformers/tree/main/examples/pytorch) Puedes encontrar estructuras de código muy similares en tareas/modelos descendentes muy diferentes usando Hugging Face.


In [ ]:
# Instalación de las bibliotecas necesarias
!pip install transformers
!pip install datasets
!pip install accelerate

Se escribe una función print_encoding diseñada para imprimir de manera legible el contenido de un diccionario,para mostrar las entradas del modelo después de la tokenización. 

In [ ]:
from collections import defaultdict, Counter
import json

from matplotlib import pyplot as plt
import numpy as np
import torch

def print_encoding(model_inputs, indent=4):
    indent_str = " " * indent
    print("{")
    for k, v in model_inputs.items():
        print(indent_str + k + ":")
        print(indent_str + indent_str + str(v))
    print("}")

### **Patrón común para usar Transformers de Hugging Face**

Vamos a empezar con un patrón de uso común para Transformadores de Hugging Face, usando el ejemplo de análisis de sentimientos.

Primero, encuentra un modelo en el [hub](https://huggingface.co/models) de Hugging Face. Cualquiera puede subir su modelo para que otras personas lo usen. (Estoy usando un modelo de análisis de sentimientos de [este artículo](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3489963)).

Luego, hay dos objetos que necesitan ser inicializados: un **tokenizador** y un **modelo**

* El tokenizador convierte cadenas en listas de IDs de vocabulario que el modelo requiere.
* El modelo toma los IDs de vocabulario y produce una predicción.

![full_nlp_pipeline.png](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)
De [https://huggingface.co/course/chapter2/2?fw=pt](https://huggingface.co/course/chapter2/2?fw=pt)



#### **RoBERTa**

RoBERTa (Robustly optimized BERT approach) es un modelo de lenguaje preentrenado desarrollado por Facebook AI. Es una variante del modelo BERT (Bidirectional Encoder Representations from Transformers) con algunas mejoras en el entrenamiento que lo hacen más robusto y eficaz en diversas tareas de procesamiento de lenguaje natural (NLP).


- Preentrenamiento con más datos: RoBERTa se entrena con más datos que BERT, lo que mejora su capacidad para capturar patrones y relaciones en el lenguaje.
- Más pasos de entrenamiento: Realiza más pasos de entrenamiento para mejorar el aprendizaje del modelo.
- Batch sizes más grandes: Utiliza lotes de datos más grandes durante el entrenamiento, lo que ayuda a estabilizar y mejorar el aprendizaje.
- Sin enmascaramiento de próxima oración: A diferencia de BERT, RoBERTa elimina la tarea de predicción de la próxima oración, lo que simplifica el entrenamiento y se enfoca más en la predicción de palabras enmascaradas.

El código siguiente utiliza el modelo RoBERTa preentrenado para la clasificación de secuencias, específicamente para la clasificación de sentimientos en inglés.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Inicializar el tokenizador
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
# Inicializar el modelo
modelo = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")


In [ ]:
inputs = "I'm  happy to learn about Hugging Face Transformers!"
tokenized_inputs = tokenizer(inputs, return_tensors="pt")
outputs = modelo(**tokenized_inputs)

labels = ['NEGATIVE', 'POSITIVE']
prediction = torch.argmax(outputs.logits)


print("Entradas:")
print(inputs)
print()
print("Entrada tokenizada:")
print_encoding(tokenized_inputs)
print()
print("Salida del modelo:")
print(outputs)
print()
print(f"La prediccion es {labels[prediction]}")

#### **Tokenizers (Tokenizadores)**

Los modelos preentrenados se implementan junto con **tokenizadores** que se usan para preprocesar sus entradas. Los tokenizadores toman cadenas de texto o listas de cadenas y producen lo que son efectivamente diccionarios que contienen las entradas del modelo.

Puedes acceder a los tokenizadores ya sea con la clase Tokenizer específica del modelo que deseas usar (aquí DistilBERT), o con la clase AutoTokenizer.
Los Fast Tokenizers están escritos en Rust, mientras que sus versiones lentas están escritas en Python.


In [ ]:
from transformers import DistilBertTokenizer, DistilBertTokenizerFast, AutoTokenizer
name = "distilbert/distilbert-base-cased"

#### **DistilBERT**

DistilBERT es una versión comprimida y optimizada del modelo BERT (Bidirectional Encoder Representations from Transformers). Fue desarrollado por Hugging Face con el objetivo de hacer que los modelos de lenguaje grandes sean más ligeros, rápidos y eficientes sin una pérdida significativa de rendimiento. DistilBERT se entrena utilizando un proceso llamado distillation (destilación), en el que un modelo más pequeño (el estudiante) aprende a reproducir el comportamiento de un modelo más grande (el maestro).

**Características principales de DistilBERT**

- Tamaño reducido: DistilBERT tiene aproximadamente la mitad de los parámetros de BERT base, lo que lo hace más ligero y fácil de desplegar en producción.
- Velocidad: Debido a su menor tamaño, DistilBERT es más rápido tanto en entrenamiento como en inferencia.
- Rendimiento: A pesar de ser más pequeño y rápido, DistilBERT mantiene alrededor del 97% de la precisión de BERT en una variedad de tareas de procesamiento de lenguaje natural.

DistilBERT se entrena utilizando un método llamado destilación de conocimiento, que implica entrenar un modelo más pequeño para imitar el comportamiento de un modelo más grande. El proceso incluye:

- Entrenamiento con pérdidas combinadas: El modelo estudiante se entrena con una combinación de la pérdida estándar (por ejemplo, pérdida de entropía cruzada) y la pérdida de distilación, que mide qué tan bien las predicciones del modelo estudiante coinciden con las del modelo maestro.
- Conservación del conocimiento: El modelo estudiante aprende a conservar y replicar el conocimiento adquirido por el modelo maestro, pero de una manera más compacta y eficiente.

El siguiente código muestra cómo inicializar y utilizar DistilBERT para tareas de tokenización:

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(name)      # escrito en Python
print(tokenizer)
tokenizer = DistilBertTokenizerFast.from_pretrained(name)  # escrito en Rust
print(tokenizer)
tokenizer = AutoTokenizer.from_pretrained(name) # por defecto es Fast
print(tokenizer)

Este resultado muestra la configuración y las propiedades de diferentes tokenizadores de DistilBERT que se han inicializado con el nombre de modelo distilbert/distilbert-base-cased.

```
DistilBertTokenizer(
    name_or_path='distilbert/distilbert-base-cased',
    vocab_size=28996,
    model_max_length=512,
    is_fast=False,
    padding_side='right',
    truncation_side='right',
    special_tokens={
        'unk_token': '[UNK]',
        'sep_token': '[SEP]',
        'pad_token': '[PAD]',
        'cls_token': '[CLS]',
        'mask_token': '[MASK]'
    },
    clean_up_tokenization_spaces=True
)
```

DistilBertTokenizerFast (dos veces con la misma configuración).

```
DistilBertTokenizerFast(
    name_or_path='distilbert/distilbert-base-cased',
    vocab_size=28996,
    model_max_length=512,
    is_fast=True,
    padding_side='right',
    truncation_side='right',
    special_tokens={
        'unk_token': '[UNK]',
        'sep_token': '[SEP]',
        'pad_token': '[PAD]',
        'cls_token': '[CLS]',
        'mask_token': '[MASK]'
    },
    clean_up_tokenization_spaces=True
)
```

El resultado muestra que has inicializado tres tokenizadores para el modelo distilbert-base-cased:

- DistilBertTokenizer: Este es el tokenizador estándar, escrito en Python, que no es tan rápido como su contraparte rápida, pero aún es ampliamente utilizado para tareas de NLP.
- DistilBertTokenizerFast (dos veces con la misma configuración): Estos son los tokenizadores rápidos, escritos en Rust, que son más eficientes en términos de tiempo de ejecución. Aunque se muestran dos veces, ambos representan la misma configuración y funcionalidad, indicando que has inicializado el tokenizador rápido más de una vez.

El código siguiente proporciona una demostración de cómo tokenizar una cadena de texto utilizando un tokenizador preentrenado de Hugging Face.

La salida incluye los identificadores de los tokens y la máscara de atención, que son esenciales para el procesamiento por parte del modelo. Además, se demuestra cómo acceder a los tokens de dos maneras diferentes, resaltando la flexibilidad de los objetos devueltos por la biblioteca transformers.

**Máscara de atención**

Una máscara de atención (attention mask) es una herramienta utilizada en modelos de procesamiento de lenguaje natural, especialmente en arquitecturas de transformers, para indicar qué tokens (palabras o subpalabras) deben ser atendidos por el modelo y cuáles deben ser ignorados durante el proceso de atención.

En el contexto de los transformers y modelos como BERT o DistilBERT, las secuencias de entrada suelen tener diferentes longitudes. Sin embargo, para procesarlas de manera eficiente en lotes (batches), las secuencias deben ser de la misma longitud. Esto se logra mediante el relleno (padding), que añade tokens especiales ([PAD]) al final de las secuencias más cortas para que todas alcancen la misma longitud. La máscara de atención se utiliza para asegurarse de que estos tokens de relleno no influyan en las predicciones del modelo.

Algunas función de la máscara de atención son:

- Indicar tokens relevantes: La máscara de atención señala qué tokens en la secuencia son relevantes y deben ser considerados por el modelo.
- Ignorar tokens de relleno: La máscara de atención asegura que los tokens de relleno ([PAD]) añadidos a las secuencias más cortas sean ignorados durante el cálculo de la atención.

La máscara de atención es una lista o tensor de la misma longitud que la secuencia de entrada tokenizada. Contiene valores binarios:

- 1: Indica que el token correspondiente es relevante y debe ser atendido.
- 0: Indica que el token correspondiente es un token de relleno y debe ser ignorado.



In [ ]:
# Así es como llamas al tokenizador
input_str = "Hugging Face Transformers is great!"
tokenized_inputs = tokenizer(input_str) # https://huggingface.co/learn/nlp-course/en/chapter6/6

print("Tokenización básica")
print_encoding(tokenized_inputs)
print()

# Dos formas de acceder:
print(tokenized_inputs.input_ids)
print(tokenized_inputs["input_ids"])

El código siguiente realiza una serie de pasos para tokenizar una cadena de texto, agregar tokens especiales (como los tokens de clasificación `[CLS]` y separación `[SEP])`, y luego decodificar los tokens de vuelta a texto.

Estos pasos no crean la máscara de atención ni añaden los caracteres especiales.

In [ ]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

# La tokenización ocurre en unos pocos pasos:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("inicio:                ", input_str)
print("tokeniza:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("agrega tokens especiales:   ", input_ids_special_tokens)
print("--------")
print("decodifica:               ", decoded_str)


El siguiente fragmento de código utiliza el FastTokenizer de la biblioteca transformers para tokenizar una cadena de texto y luego analiza los tokens resultantes en detalle. 

In [ ]:
# Para Fast Tokenizer, hay otra opción también:
inputs = tokenizer._tokenizer.encode(input_str)

print(input_str)
print("-"*5)
print(f"Número de tokens: {len(inputs)}")
print(f"Ids: {inputs.ids}")
print(f"Tokens: {inputs.tokens}")
print(f"Máscara de tokens especiales: {inputs.special_tokens_mask}")
print()
print("char_to_word da el wordpiece de un carácter en la entrada")
char_idx = 8
print(f"Por ejemplo, el {char_idx + 1}º carácter de la cadena es '{input_str[char_idx]}',"+\
      f" y es parte del wordpiece {inputs.char_to_token(char_idx)}, '{inputs.tokens[inputs.char_to_token(char_idx)]}'")


In [ ]:
# Otros trucos interesantes:
# El tokenizador puede devolver tensores de pytorch
model_inputs = tokenizer("¡Los Transformadores de Hugging Face son geniales!", return_tensors="pt")
print("Tensores PyTorch:")
print_encoding(model_inputs)

El código siguiente demuestra cómo tokenizar y rellenar múltiples secuencias de texto para que tengan la misma longitud, lo cual es necesario para el procesamiento por lotes en modelos de transformers. 

También se muestra cómo se utilizan los tokens de relleno (padding) y las máscaras de atención para indicar qué partes de las secuencias son relevantes para el modelo.

Esta técnica asegura que los modelos de lenguaje puedan procesar secuencias de longitud variable de manera eficiente y precisa.

In [ ]:
model_inputs = tokenizer(["Hugging Face Transformers is great!",
                         "The quick brown fox jumps over the lazy dog." +\
                         "Then the dog got up and ran away because she didn't like foxes.",
                         ],
                         return_tensors="pt",
                         padding=True,
                         truncation=True)
print(f"Token de relleno: {tokenizer.pad_token} | ID del token de relleno: {tokenizer.pad_token_id}")
print("Relleno (padding):")
print_encoding(model_inputs)

In [ ]:
# También puedes decodificar un lote completo a la vez:
print("Decodificación por Lote:")
print(tokenizer.batch_decode(model_inputs.input_ids))
print()
print("Decodificación por Lote: (sin caracteres especiales)")
print(tokenizer.batch_decode(model_inputs.input_ids, skip_special_tokens=True))

Para obtener más información sobre los tokenizadores, puedes consultar:
[Hugging Face Transformers Docs](https://huggingface.co/docs/transformers/main_classes/tokenizer) y la [Hugging Face Tokenizers Library](https://huggingface.co/docs/tokenizers/python/latest/quicktour.html).

>La librería de Tokenizers incluso te permite entrenar tus propios tokenizadores.


#### **Modelos**

Inicializar un modelo es muy similar a inicializar un tokenizador. Puedes elegir:

* **Clase concreta**, si conoces la arquitectura exacta que necesitas (por ejemplo, `DistilBertModel`, `BertForMaskedLM`, `GPT2Model`, etc.).
* **Clases auto-configurables** (`AutoModel*`), cuando quieras flexibilidad o comparar varios modelos especificando simplemente su nombre como cadena.

> **Recomendación:** usa `AutoModelForSequenceClassification`, `AutoModelForMaskedLM`, etc., para aprovechar el mapeo automático de heads según la tarea.

Aunque la mayoría de los Transformers comparten una arquitectura base, tienen "cabeceras" ("heads") adicionales que debes entrenar para tareas específicas, como clasificación de secuencias, extracción de entidades, preguntas y respuestas, etc. Hugging Face configura automáticamente esas heads cuando eliges la clase apropiada:

* **Clasificación de secuencias**:

  ```python
  from transformers import DistilBertForSequenceClassification
  modelo = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
  ```
* **Modelado de lenguaje enmascarado**:

  ```python
  from transformers import DistilBertForMaskedLM
  modelo = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
  ```
* **Representaciones generales**:

  ```python
  from transformers import DistilBertModel
  modelo = DistilBertModel.from_pretrained("distilbert-base-uncased")
  ```

Aquí tienes los prefijos disponibles (donde `*` puede ser `AutoModel` o el nombre de un modelo específico):

* **`*Model`**
* **`*ForMaskedLM`**
* **`*ForSequenceClassification`**
* **`*ForTokenClassification`**
* **`*ForQuestionAnswering`**
* **`*ForMultipleChoice`**

Y los tres grandes tipos de modelo:

* **Encoders** (p. ej., `BertModel`, `DistilBertModel`)
* **Decoders** (p. ej., `GPT2Model`)
* **Encoder–Decoder** (p. ej., `BartModel`, `T5Model`)

Una lista completa de clases y compatibilidades está disponible en los [docs de Transformers](https://huggingface.co/docs/transformers/model_doc/auto). Ten en cuenta que no todos los modelos soportan todas las tareas (por ejemplo, DistilBERT no es compatible con tareas Seq2Seq, ya que solo implementa un encoder).

Aquí tienes una imagen de un modelo recreada a partir de una encontrada aquí: [https://huggingface.co/course/chapter2/2?fw=pt](https://huggingface.co/course/chapter2/2?fw=pt).
![model_illustration.png](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)


In [ ]:
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification, DistilBertModel
print('Cargando el modelo base')
modelo_base = DistilBertModel.from_pretrained('distilbert-base-cased')
print("Cargando el modelo de clasificación desde el checkpoint del modelo base")
modelo = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)
modelo = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

También puedes inicializar con pesos aleatorios.

In [ ]:
from transformers import DistilBertConfig, DistilBertModel

# Inicializando una configuración de DistilBERT
configuracion = DistilBertConfig()
configuracion.num_labels=2
# Inicializando un modelo (con pesos aleatorios) desde la configuración
modelo = DistilBertForSequenceClassification(configuracion)

# Accediendo a la configuración del modelo
configuracion = modelo.config


Pasar entradas al modelo es súper fácil. Este código realiza la inferencia utilizando un modelo de clasificación de secuencias entrenado con un texto tokenizado. Aquí:

- Se convierte la cadena de entrada en tokens y los representa como tensores de PyTorch.
- Los tokens se acompañan de una máscara de atención que indica qué tokens son relevantes.
- Se realiza la inferencia usando los input_ids y attention_mask.
- El modelo produce logits que representan las salidas antes de aplicar la función softmax.
- Se aplica la función softmax a los logits para obtener probabilidades de clase.
- Se interpreta la clase más probable basada en estas probabilidades.

Este flujo de trabajo muestra cómo se utiliza un modelo de clasificación de secuencias preentrenado para hacer predicciones sobre una cadena de texto tokenizada. La salida incluye tanto los tokens de entrada como las predicciones del modelo en forma de logits y distribuciones de probabilidad.


In [ ]:
model_inputs = tokenizer(input_str, return_tensors="pt")
# Opción 1
model_outputs = modelo(input_ids=model_inputs.input_ids, attention_mask=model_inputs.attention_mask)

# Opción 2 - las claves del diccionario que devuelve el tokenizador son las mismas que los argumentos de palabra clave
#            que espera el modelo

# f({k1: v1, k2: v2}) = f(k1=v1, k2=v2)

model_outputs = modelo(**model_inputs)

print(model_inputs)
print()
print(model_outputs)
print()
print(f"Distribución sobre etiquetas: {torch.softmax(model_outputs.logits, dim=1)}")


Si te das cuenta, es un poco extraño que tengamos dos clases para una tarea de clasificación binaria - podrías fácilmente tener una sola clase y simplemente elegir un umbral. Es así por cómo los modelos de huggingface calculan la pérdida. Esto aumentará el número de parámetros que tenemos, pero no debería afectar el rendimiento.

Estos modelos son solo módulos de Pytorch. Puedes calcular la pérdida con tu `loss_func` y llamar a `loss.backward`. Puedes usar cualquiera de los optimizadores o planificadores de tasas de aprendizaje que usas.

In [ ]:
# Puedes calcular la función de pérdida
label = torch.tensor([1])
loss = torch.nn.functional.cross_entropy(model_outputs.logits, label)
print(loss)
loss.backward()
# Puedes obtener los parámetros
list(modelo.named_parameters())[0]

Hugging Face proporciona una forma adicional fácil de calcular la pérdida también:

In [ ]:
# Para calcular la pérdida, necesitamos pasar una etiqueta:
model_inputs = tokenizer(input_str, return_tensors="pt")

labels = ['NEGATIVE', 'POSITIVE']
model_inputs['labels'] = torch.tensor([1])

model_outputs = modelo(**model_inputs)

print(model_outputs)
print()
print(f"Predicciones del modelo: {labels[model_outputs.logits.argmax()]}")

Puedes obtener los estados ocultos y los pesos de atención de los modelos muy fácilmente. Esto es particularmente útil si estás trabajando en un proyecto de análisis. (Por ejemplo, ver [What does BERT look at?](https://arxiv.org/abs/1906.04341)).


El código siguiente carga un modelo preentrenado de distilbert-base-cased utilizando la biblioteca transformers de Hugging Face. Luego, el modelo es utilizado para generar salidas ocultas y atenciones para un texto de entrada.

In [ ]:
from transformers import AutoModel

modelo = AutoModel.from_pretrained("distilbert-base-cased", output_attentions=True, output_hidden_states=True)
modelo.eval()

model_inputs = tokenizer(input_str, return_tensors="pt")
with torch.no_grad():
    model_output = modelo(**model_inputs)

print("Tamaño del estado oculto (por capa):  ", model_output.hidden_states[0].shape)
print("Tamaño del head de atención (por capa):", model_output.attentions[0].shape)     # (capa, lote, índice_palabra_consulta, índices_palabra_clave)
                                                                                       # eje y es consulta, eje x es clave
# print(model_output)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(model_inputs.input_ids[0])
print(tokens)

n_layers = len(model_output.attentions)
n_heads = len(model_output.attentions[0][0])
fig, axes = plt.subplots(6, 12)
fig.set_size_inches(18.5*2, 10.5*2)
for layer in range(n_layers):
    for i in range(n_heads):
        axes[layer, i].imshow(model_output.attentions[layer][0, i])
        axes[layer][i].set_xticks(list(range(9)))
        axes[layer][i].set_xticklabels(labels=tokens, rotation="vertical")
        axes[layer][i].set_yticks(list(range(9)))
        axes[layer][i].set_yticklabels(labels=tokens)

        if layer == 5:
            axes[layer, i].set(xlabel=f"head={i}")
        if i == 0:
            axes[layer, i].set(ylabel=f"layer={layer}")

plt.subplots_adjust(wspace=0.3)
plt.show()

La salida del código es una figura  con múltiples subplots organizados en una cuadrícula de 6 filas y 12 columnas. Cada subplot representa la matriz de atención de una cabecera de atención específica en una capa específica del modelo. En cada matriz de atención:

- Eje X: Representa los tokens de la secuencia de entrada que actúan como claves.
- Eje Y: Representa los tokens de la secuencia de entrada que actúan como consultas.
- Valores: Los valores en la matriz indican cuánta atención está poniendo un token de consulta en cada token de clave. Un valor más alto indica más atención.

Este tipo de visualización es útil para entender cómo el modelo está distribuyendo su atención en diferentes partes de la secuencia de entrada a través de sus múltiples capas y cabeceras de atención.

### **Ejemplo: Mini-pipeline de fine-tuning con Transformers**

A continuación se muestra un componente explicativo, titulado que ilustra los pasos básicos para preparar un modelo para entrenamiento usando `datasets`, `DataCollator` y `Trainer` de Hugging Face.

**Mini-pipeline de fine-tuning**

1. **Cargar el dataset**
   Usamos el conjunto de reseñas de IMDb desde `datasets`.

   ```python
   from datasets import load_dataset

   raw_datasets = load_dataset("imdb", split={"train": "train[:1%]", "test": "test[:1%]"})
   ```

2. **Tokenizar**
   Inicializamos un tokenizador y aplicamos la tokenización en lote:

   ```python
   from transformers import AutoTokenizer

   tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

   def tokenize_batch(batch):
       return tokenizer(batch["text"], padding="max_length", truncation=True)

   tokenized = raw_datasets.map(tokenize_batch, batched=True)
   ```

3. **Data Collator**
   Configuramos un collator que ajuste dinámicamente el padding por batch:

   ```python
   from transformers import DataCollatorWithPadding

   data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
   ```

4. **Inicializar el modelo**
   Cargamos un modelo para clasificación de secuencias:

   ```python
   from transformers import AutoModelForSequenceClassification

   modelo = AutoModelForSequenceClassification.from_pretrained(
       "distilbert-base-uncased", num_labels=2
   )
   ```

5. **Configurar el Trainer**
   Preparamos los argumentos de entrenamiento y creamos el `Trainer`:

   ```python
   from transformers import Trainer, TrainingArguments

   training_args = TrainingArguments(
       output_dir="./results",
       per_device_train_batch_size=8,
       per_device_eval_batch_size=8,
       logging_steps=10,
       evaluation_strategy="steps",
       eval_steps=50,
   )

   trainer = Trainer(
       model=modelo,
       args=training_args,
       train_dataset=tokenized["train"],
       eval_dataset=tokenized["test"],
       data_collator=data_collator,
       tokenizer=tokenizer,
   )
   ```

> **Nota:** aquí hemos omitido el bucle de entrenamiento explícito. Para iniciar el fine-tuning bastaría con llamar `trainer.train()`.



#### **Ejemplo: Inferencia rápida con pipelines de Transformers**

**Análisis de sentimiento**

Usando el pipeline de análisis de sentimientos, en unas pocas líneas obtienes la polaridad de cualquier texto:

```python
from transformers import pipeline

# Inicializamos el pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

# Ejemplo de uso
resultados = sentiment_analyzer(
    [
        "¡Este tutorial es fantástico!",
        "No me gustó la última película que vi."
    ]
)

for res in resultados:
    print(f"Etiqueta: {res['label']}, Confianza: {res['score']:.2f}")
```

**Respuesta a preguntas**

Para responder preguntas a partir de un contexto dado, usamos el pipeline de preguntas y respuestas:

```python
from transformers import pipeline

# Inicializamos el pipeline
qa_pipeline = pipeline("question-answering")

# Contexto y pregunta
contexto = (
    "Hugging Face es una empresa que desarrolla tecnologías open-source "
    "para procesamiento de lenguaje natural. Su librería Transformers "
    "proporciona acceso a modelos como BERT, GPT y T5."
)
pregunta = "¿Qué librería proporciona acceso a modelos como BERT y GPT?"

# Ejecutamos la inferencia
respuesta = qa_pipeline(question=pregunta, context=contexto)

print(f"Respuesta: {respuesta['answer']}")
print(f"Puntaje: {respuesta['score']:.2f}")
```



### **Ejercicios**

**Ejercicio 1: Identificación de clases**

Para cada uno de estos casos, indica qué clase de modelo usarías (p. ej. `AutoModelForTokenClassification`, `BertModel`, etc.):

1. Obtener representaciones ocultas de frases para un downstream propio.
2. Entrenar un modelo para etiquetado de entidades (NER).
3. Fine-tuning de un modelo para responder preguntas tipo SQuAD.
4. Continuar preentrenamiento de BERT usando MLM.
5. Clasificar reseñas de producto en positivo/negativo.

**Entrega:** una tabla con dos columnas: "caso" y "clase recomendada".


**Ejercicio 2: Código de inicialización**

Escribe snippets de Python que carguen desde el hub el modelo `bert-base-cased` para:

a) Clasificación de secuencias
b) Modelado de lenguaje enmascarado
c) Extracción de representaciones ("feature‐extraction")

Incluye la importación y la llamada a `from_pretrained`, y comenta brevemente cuándo usarías cada uno.


**Ejercicio 3: Comparativa AutoModel vs clase específica**

1. Carga `bert-base-uncased` como `AutoModelForSequenceClassification` y como `BertForSequenceClassification`.
2. Escribe un pequeño script que imprima el número total de parámetros de cada uno y comprueba si difieren.
3. Explica por qué querrías usar una u otra en un entorno de producción.


**Ejercicio 4: Debug de heads**

Supón que al intentar entrenar:

```python
from transformers import AutoModelForMaskedLM
modelo = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
```

obtienes un error de "size mismatch" en la cabecera de MLM.

* ¿Cuál puede ser la causa?
* ¿Cómo lo resolverías sin cambiar la arquitectura base?

**Ejercicio 5: Construcción de un modelo encoder–decoder**

1. Elige un modelo Seq2Seq (p. ej. T5 o BART) y carga sus componentes encoder y decoder por separado usando las clases específicas.
2. Muestra con un snippet cómo extraer la representación del encoder y luego pasarla al decoder.
3. Describe en qué escenarios reales usarías este flujo manual en lugar de `AutoModelForSeq2SeqLM`.

**Ejercicio 6: Experimento de fine-tuning**

En un dataset pequeño (puede ser IMDB o alguno de `datasets`), realiza un fine-tuning rápido de `distilbert-base-uncased` para clasificación binaria:

1. Prepara el tokenizador y codifica el dataset.
2. Inicializa `AutoModelForSequenceClassification`.
3. Escribe el bucle de entrenamiento (al menos un epoch) con `AdamW`.
4. Reporta accuracy en validación.

**Bonus:** Compara tiempo y precisión entrenando con la clase específica (`DistilBertForSequenceClassification`) y con la auto‐configurable.

**Ejercicio 7: Visualiza los pesos de atención de un modelo preentrenado**

Instrucciones:

- Carga el modelo bert-base-uncased con la opción output_attentions=True.
- Usa una frase de tu elección y pasa por el modelo para obtener los pesos de atención.
- Visualiza los pesos de atención usando matplotlib.

In [ ]:
# Tus respuestas

### **Low-Rank Adaptation (LoRA)**


Low-Rank Adaptation (LoRA) es una técnica de *fine-tuning* eficiente que introduce matrices de baja rango en las capas de atención y feed-forward de un modelo preentrenado, en lugar de actualizar todos los parámetros.

* **Motivación**: reducir el número de parámetros entrenables y la memoria necesaria.
* **Idea clave**: en lugar de actualizar la matriz $W \in \mathbb{R}^{d \times k}$, se factoriza la actualización $\Delta W = A B$, con $A \in \mathbb{R}^{d \times r}$, $B \in \mathbb{R}^{r \times k}$ y $r \ll \min(d, k)$.

**Ventajas de LoRA**

* **Eficiencia de memoria**: solo almacenas $A$ y $B$, no todo $\Delta W$.
* **Velocidad de entrenamiento**: menos gradientes y parámetros a actualizar.
* **Compartición de pesos base**: el modelo original se mantiene intacto, facilitando combinaciones de varios LoRAs.

**Cómo aplicar LoRA con Hugging Face**


```bash
pip install peft transformers accelerate
```

**Flujo básico de trabajo**

1. **Carga de modelo y tokenizador**

   ```python
   from transformers import AutoModelForSequenceClassification, AutoTokenizer
   from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

   model_name = "bert-base-uncased"
   tokenizer = AutoTokenizer.from_pretrained(model_name)
   base_model = AutoModelForSequenceClassification.from_pretrained(model_name)
   ```

2. **Configuración de LoRA**

   ```python
   peft_config = LoraConfig(
       task_type=TaskType.SEQ_CLS,  # tipo de tarea
       r=8,                         # rango bajo
       alpha=16,                    # escala de LoRA
       target_modules=["query", "value"],  # capas a adaptar
       lora_dropout=0.05
   )
   ```

3. **Construcción del modelo LoRA**

   ```python
   lora_model = get_peft_model(base_model, peft_config)
   ```

4. **Fine-tuning**

   ```python
   from transformers import Trainer, TrainingArguments

   training_args = TrainingArguments(
       output_dir="./lora_results",
       per_device_train_batch_size=16,
       num_train_epochs=3,
       logging_steps=10,
       save_total_limit=2,
   )
   trainer = Trainer(
       model=lora_model,
       args=training_args,
       train_dataset=tokenized_train,
       eval_dataset=tokenized_eval,
       tokenizer=tokenizer,
   )
   trainer.train()
   ```

5. **Guardado y carga**

   ```python
   lora_model.save_pretrained("./lora_adapter")
   # Para reusar:
   from peft import PeftModel
   loaded = PeftModel.from_pretrained(base_model, "./lora_adapter")
   ```


#### **Cuantización (Quantization) con bitsandbytes y Optimum**


La cuantización convierte pesos de punto flotante (FP32) a representaciones de menor precisión (p. ej., INT8, INT4) para:

* **Reducir memoria**: los modelos ocupan 2-4× menos espacio.
* **Acelerar inferencia**: menos datos a mover en GPU/CPU.
* **Mantener precisión**: las bibliotecas modernas minimizan la pérdida de calidad.

**bitsandbytes (bnb)**

1. **Instalación**

   ```bash
   pip install bitsandbytes
   ```

2. **Carga de un modelo cuantizado**

   ```python
   from transformers import AutoModelForSequenceClassification, AutoTokenizer

   tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
   model_8bit = AutoModelForSequenceClassification.from_pretrained(
       "bert-base-uncased",
       load_in_8bit=True,            # activa cuantización a 8-bit
       device_map="auto"             # mapea capas a GPUs/CPU automáticamente
   )
   ```

3. **Uso en inferencia**

   ```python
   from transformers import pipeline

   sentiment = pipeline("sentiment-analysis", model=model_8bit, tokenizer=tokenizer)
   print(sentiment("La cuantización me permite ahorrar memoria."))
   ```

**Hugging Face Optimum**

Optimum es una extensión oficial de Hugging Face para optimización en hardware diverso.

1. **Instalación**

   ```bash
   pip install optimum optimum-bnb
   ```

2. **QuantizationConfig y cuantización con Optimum**

   ```python
   from optimum.bnb import BitsAndBytesConfig
   from optimum.bnb import BNBFineTuner

   bnb_config = BitsAndBytesConfig(
       load_in_4bit=True,      # o load_in_8bit
       bnb_4bit_quant_type="nf4",
       bnb_4bit_use_double_quant=True
   )

   tuner = BNBFineTuner(
       model_name_or_path="bert-base-uncased",
       quantization_config=bnb_config,
       output_dir="./bnb_quant"
   )
   tuner.quantize()
   model_4bit = tuner.get_quantized_model()
   ```

3. **Beneficios de Optimum + bnb**

   * Compatibilidad con ONNX y aceleradores especializados.
   * Pipelines automatizados para cuantización y benchmarking.
   * Integración con **Accelerate** para despliegue escalable.

Con estas técnicas, podrás entrenar y desplegar LLMs de gran tamaño de forma más eficiente en recursos y coste, manteniendo alta calidad en tus aplicaciones.


### **Uso de LLMs: Generación de texto con Llama 3 y DBRX**

A continuación tienes un ejemplo ligero de cómo cargar un modelo de generación grande (Llama 3 o DBRX) y realizar un prompt básico, optimizado para consumir pocos recursos gracias a la cuantización en 8 bits y al mapeo automático de dispositivos.

```python
from transformers import pipeline

# 1. Cargar pipeline de generación en 8-bit
generator_llama3 = pipeline(
    "text-generation",
    modelo="meta-llama/Llama-3b",        # Variante de 3 mil millones de parámetros
    tokenizer="meta-llama/Llama-3b",
    device_map="auto",                   # Asigna capas a GPU/CPU según disponibilidad
    load_in_8bit=True,                   # Cuantiza pesos a 8-bits para reducir memoria
)

# 2. Prompt de ejemplo
prompt = (
    "Eres un asistente experto en biología molecular. "
    "Explica brevemente qué es la transcripción del ADN."
)

# 3. Generación
outputs = generator_llama3(
    prompt,
    max_new_tokens=60,
    do_sample=True,
    temperature=0.7,
)

print(outputs[0]["generated_text"])
```

> **Tip de eficiencia**:
>
> * `load_in_8bit=True` ahorra \~4× en memoria.
> * `device_map="auto"` reparte el modelo entre GPU(s) y CPU para evitar OOM.
> * Ajusta `max_new_tokens` y `temperature` según necesidad de creatividad y longitud.

**DBRX**

```python
# Ejemplo similar con un modelo DBRX (reemplaza el identificador por el disponible en HF Hub)
generator_dbrx = pipeline(
    "text-generation",
    modelo="dbrx/DbrX-medium",
    tokenizer="dbrx/DbrX-medium",
    device_map="auto",
    load_in_8bit=True,
)

prompt2 = "Resume en tres líneas las ventajas de la energía solar."
out2 = generator_dbrx(prompt2, max_new_tokens=40, do_sample=False)
print(out2[0]["generated_text"])
```



#### **Ejercicios**

#### **Ejercicios LoRA**

1. **Calcular ahorro de parámetros**

   * Dado un modelo con una capa de tamaño $d\times k$, elige tres valores de rango $r$ (p. ej. 4, 16, 32).
   * Calcula cuántos parámetros adicionales introduce LoRA en cada caso y qué porcentaje representa respecto al fine-tuning completo.

2. **Selección de módulos**

   * Enumera al menos cuatro submódulos de un transformer (query, key, value, dense, etc.).
   * Para cada uno, argumenta brevemente (2-3 líneas) si lo adaptarías con LoRA para una tarea de clasificación de texto corto.

3. **Diseño de experimento sencillo**

   * Diseña un experimento de validación cruzada (por ejemplo, 3-fold) comparando fine-tuning completo vs. LoRA con $r=8$.
   * Especifica las métricas a recoger (accuracy, tiempo de entrenamiento, uso de memoria).


#### **Ejercicios de cuantización**

1. **Conversión teórica a 8-bits**

   * Explica en 3–4 líneas qué sucede con la representación de los pesos al pasar de FP32 a INT8.
   * Describe un posible efecto adverso en la calidad de inferencia.

2. **Planificación de benchmark**

   * Propón un mini-benchmark para medir la velocidad de inferencia en un modelo cuantizado vs. original (sin escribir código, solo pasos y métricas).

3. **Combinación con LoRA**

   * Esboza un flujo donde primero aplicas LoRA y luego cuantizas el modelo resultante.
   * ¿En qué orden y por qué?


#### **Ejercicios con Llama 3 y DBRX**

1. **Selección de modelo en el Hub**

   * Navega el Hugging Face Hub y elige un identificador de modelo Llama 3 y otro de DBRX.
   * Anota la configuración recomendada (parámetros, licencia, tamaño).

2. **Diseño de prompt comparativo**

   * Define dos prompts breves (uno educativo y otro creativo).
   * Explica cómo variarías `temperature` y `max_new_tokens` para cada prompt y por qué.

3. **Análisis de salida**

   * Genera dos respuestas (una con Llama 3 y otra con DBRX) para el mismo prompt.
   * Anota en una tabla comparativa diferencias en coherencia, longitud y estilo (sin código, solo describiendo).



In [ ]:
## Tus respuestas